In [1]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LEARNING_RATE = 1e-5

In [2]:
### training set 있는 Path ###
BASE_PATH = '../data/train/'

In [3]:
import torchvision.transforms as transforms
### Transforms ###
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(300)
])

## Load dataset

In [4]:
from Dataset import TrainValidDataset

In [5]:
import pandas as pd
df = pd.read_csv(BASE_PATH + 'train.csv')

In [6]:
# training set (validation set 따로 안 나눈 경우)
# train_dataset = AllClassDataset(
#     base_path = BASE_PATH, 
#     data = df, 
#     transform = transform
# )

In [7]:
# train / validation data
from sklearn.model_selection import train_test_split
def mapAgeGender(age, gender):
    answer = 0
    if age < 30:
        answer += 0
    elif age >= 60:
        answer += 2
    else:
        answer += 1
    return answer if gender == 'male' else answer + 3

y_data = df.apply(lambda x: mapAgeGender(x['age'], x['gender']), axis=1)
x_train, x_val, y_train, y_val = train_test_split(df.index, y_data, test_size=0.2, random_state=42, stratify=y_data)

In [8]:
mask_dataset = TrainValidDataset(
    base_path = BASE_PATH, 
    data = df.loc[x_train], 
    transform = transform,
    label="mask"
)

100%|██████████| 2160/2160 [00:15<00:00, 137.42it/s]


In [9]:
mask_val_dataset = TrainValidDataset(
    base_path = BASE_PATH, 
    data = df.loc[x_val], 
    transform = transform,
    label="mask"
)

100%|██████████| 540/540 [00:03<00:00, 156.58it/s]


### Dataset 확인용 Code
```python
import matplotlib.pyplot as plt
image, label = val_dataset[0]
plt.title(f'Class {label}')
plt.imshow(image.permute(1,2,0))
plt.show()
```

## DataLoader

In [10]:
mask_trainloader = torch.utils.data.DataLoader(
    mask_dataset,
    batch_size=64,
    num_workers=1
)

In [11]:
mask_valloader = torch.utils.data.DataLoader(
    mask_val_dataset,
    batch_size=64,
    num_workers=1
)

### Dataloader 확인용 Code
```python
import matplotlib.pyplot as plt
images, labels = next(iter(trainloader))
plt.title(f'Class {labels[0]}')
plt.imshow(images[0].permute(1,2,0))
plt.show()
```

## Model

In [12]:
from Model import ModifiedModel

In [13]:
mask_model = ModifiedModel(num_classes = 3).to(DEVICE)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /opt/ml/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [14]:
mask_optimizer = torch.optim.Adam(mask_model.parameters(), lr=LEARNING_RATE)

## Train

In [15]:
from train import train

In [16]:
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
train_writer = SummaryWriter('runs/train_resnet18_mask')
val_writer = SummaryWriter('runs/val_resnet18_mask')

In [17]:
### Parameter 변경 ###
train(
    model=mask_model, 
    optimizer=mask_optimizer,
    train_loader=mask_trainloader, 
    val_loader=mask_valloader,
    device=DEVICE, 
    epochs=30, 
    save=True, 
    saved_folder="saved",
    train_writer=train_writer,
    val_writer=val_writer
)

  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 001: Loss: 0.2760 / Acc: 0.915 / F1: 0.90        | Val Loss: 0.0478 / Val Acc: 0.989 / Val F1: 0.98


  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 002: Loss: 0.0251 / Acc: 0.996 / F1: 0.99        | Val Loss: 0.0214 / Val Acc: 0.994 / Val F1: 0.99


  0%|          | 0/237 [00:00<?, ?it/s]

...

Epoch 021: Loss: 0.0002 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.0047 / Val Acc: 0.998 / Val F1: 1.00


  0%|          | 0/237 [00:00<?, ?it/s]

Epoch 022: Loss: 0.0002 / Acc: 1.000 / F1: 1.00        | Val Loss: 0.0047 / Val Acc: 0.998 / Val F1: 1.00


 31%|███       | 73/237 [00:20<00:45,  3.61it/s]


KeyboardInterrupt: 

In [18]:
train_writer.close()
val_writer.close()

In [19]:
# rm -rf /opt/ml/.cache/torch/hub/checkpoints/